In [1]:
pip install gspread

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install oauth2client

     |████████████████████████████████| 102kB 3.1MB/s ta 0:00:011
     |████████████████████████████████| 102kB 16.9MB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [23]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd

In [24]:
scope = [
    'https://www.googleapis.com/auth/spreadsheets',
]

In [25]:
GOOGLE_KEY_FILE = 'gsheets_key.json'

In [26]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_KEY_FILE, scope)
gc = gspread.authorize(credentials)

In [27]:
wokbook_key = '1mGcKJNUJJvVbN4ycStlQM6PIyWe7ZX1TSSu0OEjH6eY'
workbook = gc.open_by_key(wokbook_key)

In [28]:
sheet = workbook.get_worksheet(2)
values = sheet.get_all_values()

In [29]:
covid_data = pd.DataFrame(values[1:],columns=values[0])

In [30]:
covid_data.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200407,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414,D_20200415,D_20200416
0,Adams County,18001,,,,,1,1,1,1,...,2,2,2,4,4,5,5,6,6,6
1,Allen County,18003,,,,,,,,,...,87,95,105,119,136,147,156,167,180,224
2,Bartholomew County,18005,,,,,,,,,...,55,66,83,83,108,110,110,110,113,116
3,Benton County,18007,,,,,,,,,...,0,0,0,0,0,2,2,2,3,4
4,Blackford County,18009,,,,,,,,,...,4,4,4,4,5,5,5,6,6,6


In [31]:
decatur_johnson_vanderburgh_covid = covid_data.loc[covid_data['County'].isin(['Decatur County', 'Johnson County', 'Vanderburgh County'])]
boone_lake_warrick_covid = covid_data.loc[covid_data['County'].isin(['Boone County', 'Lake County', 'Warrick County'])]

In [19]:
boone_lake_warrick_covid.head()

,County,GEOGID,d_20200306,d_20200307,d_20200308,d_20200309,d_20200310,d_20200311,d_20200312,d_20200313,...,D_20200407,D_20200408,D_20200409,D_20200410,D_20200411,D_20200412,D_20200413,D_20200414,D_20200415,D_20200416
5,Boone County,18011,,,,,1,1,1,1,...,74,76,84,90,103,109,109,125,129,136
44,Lake County,18089,,,,,,,,,...,418,461,525,576,676,744,796,819,876,978
86,Warrick County,18173,,,,,,,,,...,23,26,30,33,34,36,38,39,41,51


In [32]:
decatur_johnson_vanderburgh_covid.columns

Index(['County', 'GEOGID', 'd_20200306', 'd_20200307', 'd_20200308',
       'd_20200309', 'd_20200310', 'd_20200311', 'd_20200312', 'd_20200313',
       'd_20200314', 'd_20200315', 'd_20200316', 'd_20200317', 'd_20200318',
       'd_20200319', 'd_20200320', 'd_20200321', 'd_20200322', 'D_20200323',
       'D_20200324', 'D_20200325', 'D_20200326', 'D_20200327', 'D_20200328',
       'D_20200329', 'D_20200330', 'D_20200331', 'D_20200401', 'D_20200402',
       'D_20200403', 'D_20200404', 'D_20200405', 'D_20200406', 'D_20200407',
       'D_20200408', 'D_20200409', 'D_20200410', 'D_20200411', 'D_20200412',
       'D_20200413', 'D_20200414', 'D_20200415', 'D_20200416'],
      dtype='object')

In [33]:
djv_melt = decatur_johnson_vanderburgh_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")
blw_melt = boone_lake_warrick_covid.melt(id_vars=['County', 'GEOGID'], var_name='Date', value_name="Count")

In [34]:
djv_melt['Date'] = djv_melt['Date'].map(lambda x: x.lstrip('dD_'))
blw_melt['Date'] = blw_melt['Date'].map(lambda x: x.lstrip('dD_'))

In [35]:
blw_melt.head()

,County,GEOGID,Date,Count
0,Boone County,18011,20200306,
1,Lake County,18089,20200306,
2,Warrick County,18173,20200306,
3,Boone County,18011,20200307,
4,Lake County,18089,20200307,


In [36]:
import numpy as np
vanderburgh_covid = djv_melt.loc[djv_melt['County'].isin(['Vanderburgh County'])]
vanderburgh_covid = vanderburgh_covid.reset_index(drop=True) 
vanderburgh_covid.replace('', np.nan, inplace=True)
vanderburgh_covid.fillna(0, inplace=True)
vanderburgh_covid['Count'] = vanderburgh_covid['Count'].astype(int)

In [37]:
import pickle
vanderburgh_covid.to_pickle("vanderburgh_covid.pkl")

In [17]:
vanderburgh_covid.to_json('static/data/vanderburgh_covid.json')

In [38]:
decatur_covid = djv_melt.loc[djv_melt['County'].isin(['Decatur County'])]
decatur_covid = decatur_covid.reset_index(drop=True) 
decatur_covid.replace('', np.nan, inplace=True)
decatur_covid.fillna(0, inplace=True)
decatur_covid['Count'] = decatur_covid['Count'].astype(int)

In [39]:
decatur_covid.to_pickle("decatur_covid.pkl")

In [20]:
decatur_covid.to_json('static/data/decatur_covid.json')

In [40]:
johnson_covid = djv_melt.loc[djv_melt['County'].isin(['Johnson County'])]
johnson_covid = johnson_covid.reset_index(drop=True) 
johnson_covid.replace('', np.nan, inplace=True)
johnson_covid.fillna(0, inplace=True)
johnson_covid['Count'] = johnson_covid['Count'].astype(int)

In [41]:
johnson_covid.to_pickle("johnson_covid.pkl")

In [23]:
johnson_covid.to_json('static/data/johnson_covid.json')

In [42]:
Lake_covid = blw_melt.loc[blw_melt['County'].isin(['Lake County'])]
Lake_covid = Lake_covid.reset_index(drop=True) 
Lake_covid.replace('', np.nan, inplace=True)
Lake_covid.fillna(0, inplace=True)
Lake_covid['Count'] = Lake_covid['Count'].astype(int)

In [43]:
Lake_covid.to_pickle("Lake_covid.pkl")

In [44]:
Boone_covid = blw_melt.loc[blw_melt['County'].isin(['Boone County'])]
Boone_covid = Boone_covid.reset_index(drop=True) 
Boone_covid.replace('', np.nan, inplace=True)
Boone_covid.fillna(0, inplace=True)
Boone_covid['Count'] = Boone_covid['Count'].astype(int)

In [45]:
Boone_covid.to_pickle("Boone_covid.pkl")

In [46]:
Warrick_covid = blw_melt.loc[blw_melt['County'].isin(['Warrick County'])]
Warrick_covid = Warrick_covid.reset_index(drop=True) 
Warrick_covid.replace('', np.nan, inplace=True)
Warrick_covid.fillna(0, inplace=True)
Warrick_covid['Count'] = Warrick_covid['Count'].astype(int)

In [47]:
Warrick_covid.to_pickle("Warrick_covid.pkl")